## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-01-02-18-54-08 +0000,nyt,A Diminished Congress Weighs Whether to Reasse...,https://www.nytimes.com/2026/01/02/us/politics...
1,2026-01-02-18-53-05 +0000,nypost,Heroic off-duty ICE agent guns down active sho...,https://nypost.com/2026/01/02/us-news/off-duty...
2,2026-01-02-18-52-24 +0000,nyt,Swiss Alps Ski Resort Fire: What We Know,https://www.nytimes.com/2026/01/01/world/europ...
3,2026-01-02-18-50-38 +0000,nyt,Live Updates: Swiss Authorities Say Sparklers ...,https://www.nytimes.com/live/2026/01/02/world/...
4,2026-01-02-18-48-00 +0000,wsj,Saks CEO Steps Down Ahead of Expected Bankrupt...,https://www.wsj.com/business/retail/saks-ceo-s...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2343/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
26,new,34
27,year,30
19,fire,24
15,swiss,20
242,trump,18


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
202,2026-01-01-22-55-18 +0000,wapo,"About 40 killed, 115 injured in New Year’s bar...",https://www.washingtonpost.com/world/2026/01/0...,141
148,2026-01-02-08-05-41 +0000,nypost,Swiss face painful task of identifying victims...,https://nypost.com/2026/01/02/world-news/swiss...,139
57,2026-01-02-16-58-00 +0000,wsj,Champagne Sparklers Likely Ignited Ceiling in ...,https://www.wsj.com/world/europe/switzerland-s...,124
193,2026-01-02-01-34-44 +0000,nyt,Fire in Swiss Alps Leaves Dozens of New Year’s...,https://www.nytimes.com/2026/01/01/world/europ...,122
3,2026-01-02-18-50-38 +0000,nyt,Live Updates: Swiss Authorities Say Sparklers ...,https://www.nytimes.com/live/2026/01/02/world/...,120


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
202,141,2026-01-01-22-55-18 +0000,wapo,"About 40 killed, 115 injured in New Year’s bar...",https://www.washingtonpost.com/world/2026/01/0...
144,45,2026-01-02-08-56-35 +0000,nypost,Trump vows US ‘locked and loaded’ to help Iran...,https://nypost.com/2026/01/02/world-news/trump...
162,40,2026-01-02-05-51-10 +0000,nypost,Kim Jong Un’s daughter visits family mausoleum...,https://nypost.com/2026/01/02/world-news/kim-j...
31,39,2026-01-02-17-39-02 +0000,nypost,Zelensky taps popular Ukraine spy chief to run...,https://nypost.com/2026/01/02/world-news/zelen...
116,34,2026-01-02-11-13-20 +0000,cbc,Sparkling flares in champagne bottles coming t...,https://www.cbc.ca/news/world/swiss-fire-bar-9...
221,30,2026-01-01-20-11-45 +0000,bbc,Social media fuelled Mamdani's rise. Can he ke...,https://www.bbc.com/news/articles/c4gvg46dy7vo...
77,28,2026-01-02-15-03-38 +0000,bbc,China's BYD overtakes Tesla as world's top EV ...,https://www.bbc.com/news/articles/cj9rjwpvmpzo...
48,25,2026-01-02-17-12-51 +0000,nypost,"Tommy Lee Jones’ daughter, Victoria, was a ‘pr...",https://nypost.com/2026/01/02/us-news/tommy-le...
47,24,2026-01-02-17-16-17 +0000,nypost,Israeli tech billionaire Shlomo Kramer urges A...,https://nypost.com/2026/01/02/business/israeli...
34,24,2026-01-02-17-36-17 +0000,wapo,Woman dies in suspected mountain lion attack i...,https://www.washingtonpost.com/nation/2026/01/...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
